Voici un gabarit complet pour la partie 4 : Analyse par graphe.
Vous pourrez :

Construire un graphe biparti
Option A : Parfumeur ↔ Parfum (illustré ci-dessous)
Option B : Accord ↔ Parfum (même logique, il suffit de changer les colonnes)

Calculer : degré, betweenness, eigenvector centrality

Détecter les communautés Louvain (si python-louvain installé)

Visualiser un sous-graphe annoté pour commenter les « hubs »

Hypothèses

DataFrame : FDS

Colonnes : nose_reduit (parfumeur réduit) et nom_parfum (ou un ID)

Vous avez déjà un encodage « top k vs. autres » pour éviter 20 000 nœuds inutiles.

In [ ]:
# =========================================================
# 4.1  Grille de préparation : bipartite Parfumeur ↔ Parfum
# =========================================================
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# --- limitons-nous aux N parfumeurs les plus prolifiques ---
TOP_N = 30
top_noses = (FDS["nose_reduit"]
             .value_counts()
             .head(TOP_N)
             .index)

sub = FDS[FDS["nose_reduit"].isin(top_noses)].copy()

print(f"{len(sub)} parfums couverts par les {TOP_N} parfumeurs.")

# --- construire le graphe biparti -------------------------
B = nx.Graph()
B.add_nodes_from(sub["nom_parfum"], bipartite="parfum")
B.add_nodes_from(sub["nose_reduit"],  bipartite="nose")

edges = list(zip(sub["nose_reduit"], sub["nom_parfum"]))
B.add_edges_from(edges)
print(f"{B.number_of_nodes()} nœuds, {B.number_of_edges()} arêtes")


In [ ]:
# ========================================
# 4.2  Mesures de centralité   (NetworkX)
# ========================================
# centralité degré
deg_nose   = nx.degree_centrality(B)
deg_sorted = sorted([(n,v) for n,v in deg_nose.items()
                     if B.nodes[n]["bipartite"]=="nose"],
                    key=lambda x: x[1], reverse=True)[:10]
print("Top 10 parfumeurs (degree):")
for n, v in deg_sorted:
    print(f"{n:<20} {v:.3f}")

# betweenness (plus coûteux → petit graphe conseillé)
btw_nose = nx.betweenness_centrality(B, k=None, normalized=True,
                                     endpoints=False)
btw_sorted = sorted([(n,v) for n,v in btw_nose.items()
                     if B.nodes[n]["bipartite"]=="nose"],
                    key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 parfumeurs (betweenness):")
for n, v in btw_sorted:
    print(f"{n:<20} {v:.3f}")


In [ ]:
# =============================================
# 4.3  Détection de communautés (Louvain)
#      pip install python-louvain  si besoin
# =============================================
try:
    import community as community_louvain
    partition = community_louvain.best_partition(B)
    # ajoute les labels de communauté au graphe
    nx.set_node_attributes(B, partition, "community")
    n_communities = len(set(partition.values()))
    print(f"{n_communities} communautés Louvain détectées.")
except ImportError:
    print("⚠️  python-louvain manquant → pip install python-louvain")
    partition = None


In [ ]:
# ===========================================
# 4.4  Visualisation d'un sous-graphe réduit
#       (parfumeurs + 2 parfums associés)
# ===========================================
# choisir M parfumeurs influents
M = 8
hubs = [n for n,_ in deg_sorted[:M]]

nodes_to_keep = []
for nose in hubs:
    nodes_to_keep.append(nose)
    # prendre 2 parfums connectés (ou plus si vous voulez)
    voisins = [nbr for nbr in B.neighbors(nose)][:2]
    nodes_to_keep.extend(voisins)

G_sub = B.subgraph(nodes_to_keep)

# mise en page force-directed
pos = nx.spring_layout(G_sub, seed=0)

plt.figure(figsize=(8,6))
colors = ["#ff7f0e" if B.nodes[n]["bipartite"]=="nose" else "#1f77b4"
          for n in G_sub.nodes()]
sizes  = [300 if B.nodes[n]["bipartite"]=="nose" else 80
          for n in G_sub.nodes()]

nx.draw_networkx_nodes(G_sub, pos, node_color=colors, node_size=sizes)
nx.draw_networkx_edges(G_sub, pos, alpha=.4)
nx.draw_networkx_labels(G_sub, pos,
                        labels={n:n for n in G_sub if B.nodes[n]["bipartite"]=="nose"},
                        font_size=9, font_weight="bold")
plt.title("Mini-réseau Parfumeurs ↔ Parfums (hubs)")
plt.axis("off")
plt.tight_layout()
plt.show()


## Commentaire à rédiger (guide) 


Insight possible | Commentaire
Parfumeurs “hubs” (degré élevé) | « X » et « Y » signent respectivement 8 % et 6 % des 20 000 parfums ; ils sont donc des acteurs majeurs.
Betweenness élevé | Un parfumeur qui relie plusieurs communautés (par ex. “nez maison” d’une marque + collaborations externes) est un pivot créatif.
Communautés Louvain | Souvent organisées par marque : les parfums Dior/Chanel forment des sous-graphes denses, distincts des marques niche.
Graphe Accord ↔ Parfum (variante) | Les accords « floral » et « woody » sont hubs → confirment la dominance vue dans l’EDA ; accords rares (ex. “ozonic”) apparaissent en feuilles périphériques.

## Variante  « Accord ↔ Parfum »

In [ ]:
accord_cols = [c for c in FDS.columns if c.startswith("accord_")]
# transformer en liste (accord, parfum) pour accords actifs
edges_acc = []
for col in accord_cols:
    acc_name = col.replace("accord_", "")
    parfums = FDS[FDS[col]==1]["nom_parfum"]
    for p in parfums:
        edges_acc.append((acc_name, p))

B2 = nx.Graph()
acc_nodes = [col.replace("accord_", "") for col in accord_cols]
B2.add_nodes_from(acc_nodes, bipartite="accord")
B2.add_nodes_from(FDS["nom_parfum"], bipartite="parfum")
B2.add_edges_from(edges_acc)


## Bonne pratique

Bonnes pratiques
Limitez la taille du graphe pour la visualisation : top-30 parfumeurs ou accords, ou un échantillon de parfums.

Sauvegardez les métriques (degree, betweenness) dans un CSV ; elles pourront nourrir vos modèles prédictifs (ex. “un parfum signé par un nez hub obtient-il une meilleure note ?”).

Expliquez dans le rapport la pertinence d’analyser le réseau : le monde de la parfumerie repose sur des collaborations multiples, que les tableaux plats ne révèlent pas.

Vous avez ainsi un network analysis complet : construction, métriques, communautés, et visualisation — prêt à enrichir l’interprétation stratégique de votre projet.